In [57]:
import config
import ast
import pickle
import pandas as pd
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import statistics

from utils.pandas_dataframe import grid_display
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from itertools import permutations
import gc

# validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor

# grid search
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV


from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor

from IPython.display import display


%matplotlib inline

In [58]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format',lambda x: '%.2f' % x)
pd.set_option('display.max_rows', None)

In [59]:
def Col_types(Data):
    Column_types = Data.dtypes.to_frame().reset_index()
    Column_types.columns = ['ColumnName', 'Type']
    Column_types.sort_values( by= 'Type', inplace = True)
    return Column_types

def Missing_Counts(Data):
    missing = Data.isnull().sum()
    missing = missing[ missing >0]
    missing.sort_values( inplace=True)
    Missing_Count = pd.DataFrame({ 'ColumnName':missing.index, 'MissingCount':missing.values})
    Missing_Count['Percentage(%)'] = Missing_Count['MissingCount'].apply(lambda x:round(x/Data.shape[0]*100,2))
    return Missing_Count

In [60]:
with open('../Data/meta/train_x_after_feature_engineering.pickle', 'rb') as f:
    data_x = pickle.load(f)
data_x.head()

,clean_pressure31,clean_pressure33,clean_pressure41,clean_pressure51,clean_pressure52,clean_pressure61,clean_pressure71,clean_pressure72,clean_pressure81,clean_pressure91,...,clean_pressure82_group,oven_pb1_group,oven_pb2_group,oven_a1_group,oven_b1_group,oven_b3_group,painting_g3_act_hvv_group,painting_g8_act_f_air_group,painting_g9_act_hvc_group,env_rpi05_temp_group
0,-0.85,-2.27,2.33,2.16,-2.97,1.90,0.43,2.90,1.94,-0.43,...,5,5,3,2,2,1,3,0,4,1
1,-0.85,-2.27,2.33,2.17,-2.97,1.90,0.43,2.91,1.95,-0.47,...,5,5,3,2,2,1,3,0,4,1
4,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,5,5,3,2,2,1,3,0,1,1
5,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,5,5,3,2,2,1,3,0,1,1
6,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,5,5,3,2,2,1,3,0,1,1


In [61]:
with open('../Data/meta/train_y_after_feature_engineering.pickle', 'rb') as f:
    data_y = pickle.load(f)
data_y.head()

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.88,-1.37,-1.11,-1.66,-0.75,-2.80
1,-3.26,-1.22,-2.05,-1.71,-1.20,-2.97
4,-1.72,-0.54,-1.90,-1.23,-1.20,-1.60
5,-2.88,-0.77,-1.03,-1.10,-1.60,-1.66
6,-2.41,-0.77,-0.71,-1.66,-0.60,-2.62


In [69]:
# onehot encoding
for index , col in enumerate(data_x.columns.tolist()):
    if data_x[col].dtype == 'object':
        data_x = data_x.join(pd.get_dummies(data_x[col],prefix=col))
        data_x = data_x.drop(col, axis = 1)

In [70]:
display(Col_types(data_x))

,ColumnName,Type
51,clean_pressure82_group_1,uint8
75,oven_b1_group_1,uint8
74,oven_b1_group_0,uint8
73,oven_a1_group_5,uint8
72,oven_a1_group_4,uint8
71,oven_a1_group_3,uint8
70,oven_a1_group_2,uint8
69,oven_a1_group_1,uint8
68,oven_a1_group_0,uint8
67,oven_pb2_group_5,uint8


In [64]:
Lasso_params = {'sensor_point5_i_value': {'alpha': 500}, 
                'sensor_point6_i_value': {'alpha': 100}, 
                'sensor_point7_i_value': {'alpha': 500}, 
                'sensor_point8_i_value': {'alpha': 500}, 
                'sensor_point9_i_value': {'alpha': 500}, 
                'sensor_point10_i_value': {'alpha': 500}}

Ridge_params = {'sensor_point5_i_value': {'alpha': 0.1}, 
                'sensor_point6_i_value': {'alpha': 0.1}, 
                'sensor_point7_i_value': {'alpha': 0.1}, 
                'sensor_point8_i_value': {'alpha': 0.1}, 
                'sensor_point9_i_value': {'alpha': 1}, 
                'sensor_point10_i_value': {'alpha': 0.5}}

XGB_params = {'sensor_point5_i_value': {'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 30, 'random_state': 1}, 
              'sensor_point6_i_value': {'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 30, 'random_state': 1}, 
              'sensor_point7_i_value': {'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 500, 'random_state': 1}, 
              'sensor_point8_i_value': {'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 30, 'random_state': 1}, 
              'sensor_point9_i_value': {'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500, 'random_state': 1}, 
              'sensor_point10_i_value': {'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'n_estimators': 30, 'random_state': 1}}

KN_params = {'sensor_point5_i_value': {'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}, 
             'sensor_point6_i_value': {'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}, 
             'sensor_point7_i_value': {'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}, 
             'sensor_point8_i_value': {'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}, 
             'sensor_point9_i_value': {'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}, 
             'sensor_point10_i_value': {'leaf_size': 20, 'n_neighbors': 7, 'weights': 'distance'}}

Ada_params = {'sensor_point5_i_value': {'learning_rate': 0.2, 'n_estimators': 100, 'random_state': 1}, 
              'sensor_point6_i_value': {'learning_rate': 0.2, 'n_estimators': 100, 'random_state': 1}, 
              'sensor_point7_i_value': {'learning_rate': 0.5, 'n_estimators': 100, 'random_state': 1}, 
              'sensor_point8_i_value': {'learning_rate': 0.5, 'n_estimators': 300, 'random_state': 1}, 
              'sensor_point9_i_value': {'learning_rate': 0.2, 'n_estimators': 300, 'random_state': 1}, 
              'sensor_point10_i_value': {'learning_rate': 0.2, 'n_estimators': 100, 'random_state': 1}}

Cat_params = {'sensor_point5_i_value': {'depth': 8, 'eval_metric': 'RMSE', 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.03, 'random_state': 1}, 
              'sensor_point6_i_value': {'depth': 2, 'eval_metric': 'RMSE', 'iterations': 1000, 'l2_leaf_reg': 0.5, 'learning_rate': 0.03, 'random_state': 1}, 
              'sensor_point7_i_value': {'depth': 5, 'eval_metric': 'RMSE', 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.03, 'random_state': 1}, 
              'sensor_point8_i_value': {'depth': 5, 'eval_metric': 'RMSE', 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.03, 'random_state': 1}, 
              'sensor_point9_i_value': {'depth': 8, 'eval_metric': 'RMSE', 'iterations': 500, 'l2_leaf_reg': 0.5, 'learning_rate': 0.03, 'random_state': 1}, 
              'sensor_point10_i_value': {'depth': 8, 'eval_metric': 'RMSE', 'iterations': 500, 'l2_leaf_reg': 0.5, 'learning_rate': 0.03, 'random_state': 1}}


In [66]:
order_list = [5, 3, 2, 1, 4, 0]

model_list = [Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('Cat', CatBoostRegressor(**Cat_params[config.data_y_col[0]]))
                        ]),
                       
              Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('Cat', CatBoostRegressor(**Cat_params[config.data_y_col[1]]))
                        ]),
              
              Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('KNR', KNeighborsRegressor(**KN_params[config.data_y_col[2]]))
                        ]),
              
              Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('KNR', KNeighborsRegressor(**KN_params[config.data_y_col[3]]))
                        ]),
               
              Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('Ada', AdaBoostRegressor(**Ada_params[config.data_y_col[4]]))
                        ]),
              
              Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('Cat', CatBoostRegressor(**Cat_params[config.data_y_col[5]]))
                        ])
              ]

In [74]:
X_PT

,clean_pressure31,clean_pressure33,clean_pressure41,clean_pressure51,clean_pressure52,clean_pressure61,clean_pressure71,clean_pressure72,clean_pressure81,clean_pressure91,...,painting_g9_act_hvc_group_2,painting_g9_act_hvc_group_3,painting_g9_act_hvc_group_4,painting_g9_act_hvc_group_5,env_rpi05_temp_group_0,env_rpi05_temp_group_1,env_rpi05_temp_group_2,env_rpi05_temp_group_3,env_rpi05_temp_group_4,sensor_point10_i_value
0,-0.85,-2.27,2.33,2.16,-2.97,1.90,0.43,2.90,1.94,-0.43,...,0,0,1,0,0,1,0,0,0,-2.71
1,-0.85,-2.27,2.33,2.17,-2.97,1.90,0.43,2.91,1.95,-0.47,...,0,0,1,0,0,1,0,0,0,-2.76
4,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,0,0,0,0,0,1,0,0,0,-2.06
5,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,0,0,0,0,0,1,0,0,0,-2.06
6,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,0,0,0,0,0,1,0,0,0,-2.06
7,-0.86,-2.30,2.33,2.17,-2.96,1.90,0.38,2.90,1.81,-0.56,...,0,0,0,0,0,1,0,0,0,-2.06
8,-0.86,-2.31,2.33,2.14,-2.97,1.90,0.32,2.87,1.88,-0.44,...,0,0,1,0,0,1,0,0,0,-2.59
9,-0.86,-2.31,2.33,2.14,-2.97,1.90,0.32,2.87,1.88,-0.44,...,0,0,1,0,0,1,0,0,0,-2.59
10,-0.87,-2.33,2.33,2.01,-2.98,1.90,0.29,2.85,1.72,-0.55,...,0,0,1,0,0,1,0,0,0,-2.59
11,-0.87,-2.33,2.33,2.01,-2.98,1.90,0.29,2.85,1.72,-0.55,...,0,0,1,0,0,1,0,0,0,-2.59


In [75]:
X_PT = data_x.copy()
predict_y_train = pd.DataFrame({})

for k in range(6):
    # 按照排列順序訓練模型
    globals()['model_Y' + str(order_list[k])]= model_list[order_list[k]]
    globals()['model_Y' + str(order_list[k])].fit(X_PT, data_y.iloc[:,order_list[k]])
             
    # 儲存預測結果
    predict_y_train[config.data_y_col[order_list[k]]] = globals()['model_Y' + str(order_list[k])].predict(X_PT)
        
    # 把取得的預測值當作變數放進X
    X_PT[config.data_y_col[order_list[k]]] = globals()['model_Y' + str(order_list[k])].predict(X_PT)

    
# 調整欄位順序
predict_y_train = predict_y_train[config.data_y_col]

display(predict_y_train)

0:	learn: 0.9854193	total: 868ms	remaining: 7m 12s
1:	learn: 0.9743141	total: 1.56s	remaining: 6m 29s
2:	learn: 0.9609285	total: 2.26s	remaining: 6m 14s
3:	learn: 0.9478013	total: 2.97s	remaining: 6m 8s
4:	learn: 0.9361146	total: 3.68s	remaining: 6m 4s
5:	learn: 0.9246218	total: 4.37s	remaining: 5m 59s
6:	learn: 0.9142379	total: 5.05s	remaining: 5m 56s
7:	learn: 0.9022006	total: 5.75s	remaining: 5m 53s
8:	learn: 0.8906230	total: 6.42s	remaining: 5m 50s
9:	learn: 0.8796015	total: 7.09s	remaining: 5m 47s
10:	learn: 0.8694460	total: 7.78s	remaining: 5m 45s
11:	learn: 0.8597620	total: 8.47s	remaining: 5m 44s
12:	learn: 0.8515502	total: 9.22s	remaining: 5m 45s
13:	learn: 0.8425195	total: 10s	remaining: 5m 47s
14:	learn: 0.8337036	total: 10.8s	remaining: 5m 49s
15:	learn: 0.8253478	total: 11.6s	remaining: 5m 49s
16:	learn: 0.8165967	total: 12.3s	remaining: 5m 50s
17:	learn: 0.8107541	total: 13s	remaining: 5m 49s
18:	learn: 0.8016675	total: 13.8s	remaining: 5m 49s
19:	learn: 0.7929767	total: 

157:	learn: 0.5285665	total: 1m 55s	remaining: 4m 10s
158:	learn: 0.5280418	total: 1m 56s	remaining: 4m 9s
159:	learn: 0.5274756	total: 1m 57s	remaining: 4m 9s
160:	learn: 0.5270486	total: 1m 58s	remaining: 4m 8s
161:	learn: 0.5265695	total: 1m 58s	remaining: 4m 8s
162:	learn: 0.5259042	total: 1m 59s	remaining: 4m 7s
163:	learn: 0.5254428	total: 2m	remaining: 4m 6s
164:	learn: 0.5251398	total: 2m	remaining: 4m 5s
165:	learn: 0.5244158	total: 2m 1s	remaining: 4m 4s
166:	learn: 0.5240437	total: 2m 2s	remaining: 4m 4s
167:	learn: 0.5236422	total: 2m 3s	remaining: 4m 3s
168:	learn: 0.5230674	total: 2m 3s	remaining: 4m 2s
169:	learn: 0.5226029	total: 2m 4s	remaining: 4m 1s
170:	learn: 0.5223747	total: 2m 5s	remaining: 4m 1s
171:	learn: 0.5220173	total: 2m 6s	remaining: 4m
172:	learn: 0.5217645	total: 2m 6s	remaining: 3m 59s
173:	learn: 0.5215658	total: 2m 7s	remaining: 3m 58s
174:	learn: 0.5209981	total: 2m 8s	remaining: 3m 58s
175:	learn: 0.5207521	total: 2m 8s	remaining: 3m 57s
176:	learn

311:	learn: 0.4759491	total: 3m 48s	remaining: 2m 17s
312:	learn: 0.4757503	total: 3m 48s	remaining: 2m 16s
313:	learn: 0.4754978	total: 3m 49s	remaining: 2m 15s
314:	learn: 0.4752318	total: 3m 50s	remaining: 2m 15s
315:	learn: 0.4750706	total: 3m 50s	remaining: 2m 14s
316:	learn: 0.4748750	total: 3m 51s	remaining: 2m 13s
317:	learn: 0.4746266	total: 3m 52s	remaining: 2m 12s
318:	learn: 0.4741307	total: 3m 52s	remaining: 2m 12s
319:	learn: 0.4738231	total: 3m 53s	remaining: 2m 11s
320:	learn: 0.4736222	total: 3m 54s	remaining: 2m 10s
321:	learn: 0.4734618	total: 3m 55s	remaining: 2m 9s
322:	learn: 0.4732350	total: 3m 55s	remaining: 2m 9s
323:	learn: 0.4728924	total: 3m 56s	remaining: 2m 8s
324:	learn: 0.4727751	total: 3m 57s	remaining: 2m 7s
325:	learn: 0.4725133	total: 3m 58s	remaining: 2m 7s
326:	learn: 0.4723147	total: 3m 59s	remaining: 2m 6s
327:	learn: 0.4720137	total: 3m 59s	remaining: 2m 5s
328:	learn: 0.4718938	total: 4m	remaining: 2m 4s
329:	learn: 0.4716029	total: 4m 1s	remai

466:	learn: 0.4507372	total: 5m 39s	remaining: 24s
467:	learn: 0.4506710	total: 5m 40s	remaining: 23.3s
468:	learn: 0.4506338	total: 5m 41s	remaining: 22.5s
469:	learn: 0.4505819	total: 5m 41s	remaining: 21.8s
470:	learn: 0.4504818	total: 5m 42s	remaining: 21.1s
471:	learn: 0.4504308	total: 5m 43s	remaining: 20.4s
472:	learn: 0.4503583	total: 5m 43s	remaining: 19.6s
473:	learn: 0.4503219	total: 5m 44s	remaining: 18.9s
474:	learn: 0.4502547	total: 5m 45s	remaining: 18.2s
475:	learn: 0.4501871	total: 5m 45s	remaining: 17.4s
476:	learn: 0.4501061	total: 5m 46s	remaining: 16.7s
477:	learn: 0.4500386	total: 5m 47s	remaining: 16s
478:	learn: 0.4499811	total: 5m 48s	remaining: 15.3s
479:	learn: 0.4499362	total: 5m 48s	remaining: 14.5s
480:	learn: 0.4498508	total: 5m 49s	remaining: 13.8s
481:	learn: 0.4497814	total: 5m 50s	remaining: 13.1s
482:	learn: 0.4496863	total: 5m 51s	remaining: 12.4s
483:	learn: 0.4496159	total: 5m 51s	remaining: 11.6s
484:	learn: 0.4495205	total: 5m 52s	remaining: 10.

131:	learn: 0.4582151	total: 3.73s	remaining: 24.5s
132:	learn: 0.4576632	total: 3.76s	remaining: 24.5s
133:	learn: 0.4570544	total: 3.79s	remaining: 24.5s
134:	learn: 0.4562643	total: 3.82s	remaining: 24.5s
135:	learn: 0.4554559	total: 3.84s	remaining: 24.4s
136:	learn: 0.4547270	total: 3.87s	remaining: 24.4s
137:	learn: 0.4534261	total: 3.9s	remaining: 24.4s
138:	learn: 0.4527244	total: 3.93s	remaining: 24.3s
139:	learn: 0.4523317	total: 3.96s	remaining: 24.3s
140:	learn: 0.4517468	total: 3.99s	remaining: 24.3s
141:	learn: 0.4511221	total: 4.01s	remaining: 24.3s
142:	learn: 0.4502732	total: 4.04s	remaining: 24.2s
143:	learn: 0.4494604	total: 4.07s	remaining: 24.2s
144:	learn: 0.4487729	total: 4.1s	remaining: 24.2s
145:	learn: 0.4484273	total: 4.12s	remaining: 24.1s
146:	learn: 0.4476268	total: 4.15s	remaining: 24.1s
147:	learn: 0.4465593	total: 4.18s	remaining: 24.1s
148:	learn: 0.4461905	total: 4.21s	remaining: 24s
149:	learn: 0.4453988	total: 4.24s	remaining: 24s
150:	learn: 0.4442

...

942:	learn: 0.3166196	total: 29.3s	remaining: 1.77s
943:	learn: 0.3165873	total: 29.4s	remaining: 1.74s
944:	learn: 0.3165334	total: 29.4s	remaining: 1.71s
945:	learn: 0.3164956	total: 29.4s	remaining: 1.68s
946:	learn: 0.3164674	total: 29.5s	remaining: 1.65s
947:	learn: 0.3164058	total: 29.5s	remaining: 1.62s
948:	learn: 0.3163800	total: 29.5s	remaining: 1.58s
949:	learn: 0.3163088	total: 29.5s	remaining: 1.55s
950:	learn: 0.3162813	total: 29.6s	remaining: 1.52s
951:	learn: 0.3162221	total: 29.6s	remaining: 1.49s
952:	learn: 0.3162079	total: 29.6s	remaining: 1.46s
953:	learn: 0.3161380	total: 29.7s	remaining: 1.43s
954:	learn: 0.3160980	total: 29.7s	remaining: 1.4s
955:	learn: 0.3160627	total: 29.7s	remaining: 1.37s
956:	learn: 0.3159987	total: 29.7s	remaining: 1.34s
957:	learn: 0.3159599	total: 29.8s	remaining: 1.3s
958:	learn: 0.3159264	total: 29.8s	remaining: 1.27s
959:	learn: 0.3158809	total: 29.8s	remaining: 1.24s
960:	learn: 0.3158017	total: 29.9s	remaining: 1.21s
961:	learn: 0.

101:	learn: 0.5516468	total: 1m 33s	remaining: 6m 2s
102:	learn: 0.5506769	total: 1m 33s	remaining: 6m 1s
103:	learn: 0.5497080	total: 1m 34s	remaining: 6m
104:	learn: 0.5486730	total: 1m 35s	remaining: 5m 59s
105:	learn: 0.5479123	total: 1m 36s	remaining: 5m 58s
106:	learn: 0.5475283	total: 1m 37s	remaining: 5m 57s
107:	learn: 0.5465846	total: 1m 38s	remaining: 5m 57s
108:	learn: 0.5457290	total: 1m 39s	remaining: 5m 56s
109:	learn: 0.5447559	total: 1m 40s	remaining: 5m 55s
110:	learn: 0.5435666	total: 1m 41s	remaining: 5m 54s
111:	learn: 0.5426631	total: 1m 41s	remaining: 5m 53s
112:	learn: 0.5414555	total: 1m 42s	remaining: 5m 52s
113:	learn: 0.5401801	total: 1m 43s	remaining: 5m 51s
114:	learn: 0.5394173	total: 1m 44s	remaining: 5m 50s
115:	learn: 0.5384926	total: 1m 45s	remaining: 5m 49s
116:	learn: 0.5376633	total: 1m 46s	remaining: 5m 48s
117:	learn: 0.5369552	total: 1m 47s	remaining: 5m 47s
118:	learn: 0.5361761	total: 1m 48s	remaining: 5m 45s
119:	learn: 0.5354923	total: 1m 48

254:	learn: 0.4729682	total: 3m 46s	remaining: 3m 37s
255:	learn: 0.4725889	total: 3m 47s	remaining: 3m 36s
256:	learn: 0.4723046	total: 3m 47s	remaining: 3m 35s
257:	learn: 0.4720567	total: 3m 48s	remaining: 3m 34s
258:	learn: 0.4717812	total: 3m 49s	remaining: 3m 33s
259:	learn: 0.4715173	total: 3m 50s	remaining: 3m 32s
260:	learn: 0.4712407	total: 3m 51s	remaining: 3m 31s
261:	learn: 0.4710847	total: 3m 52s	remaining: 3m 30s
262:	learn: 0.4709450	total: 3m 53s	remaining: 3m 30s
263:	learn: 0.4705516	total: 3m 53s	remaining: 3m 29s
264:	learn: 0.4703119	total: 3m 54s	remaining: 3m 28s
265:	learn: 0.4698869	total: 3m 55s	remaining: 3m 27s
266:	learn: 0.4695927	total: 3m 56s	remaining: 3m 26s
267:	learn: 0.4691638	total: 3m 57s	remaining: 3m 25s
268:	learn: 0.4690034	total: 3m 58s	remaining: 3m 24s
269:	learn: 0.4687134	total: 3m 59s	remaining: 3m 24s
270:	learn: 0.4683718	total: 4m	remaining: 3m 23s
271:	learn: 0.4679203	total: 4m 1s	remaining: 3m 22s
272:	learn: 0.4674969	total: 4m 2

407:	learn: 0.4242060	total: 6m 5s	remaining: 1m 22s
408:	learn: 0.4240295	total: 6m 6s	remaining: 1m 21s
409:	learn: 0.4237122	total: 6m 7s	remaining: 1m 20s
410:	learn: 0.4234606	total: 6m 8s	remaining: 1m 19s
411:	learn: 0.4230567	total: 6m 9s	remaining: 1m 18s
412:	learn: 0.4227424	total: 6m 9s	remaining: 1m 17s
413:	learn: 0.4225678	total: 6m 10s	remaining: 1m 17s
414:	learn: 0.4222025	total: 6m 11s	remaining: 1m 16s
415:	learn: 0.4218452	total: 6m 12s	remaining: 1m 15s
416:	learn: 0.4216891	total: 6m 13s	remaining: 1m 14s
417:	learn: 0.4214637	total: 6m 14s	remaining: 1m 13s
418:	learn: 0.4212788	total: 6m 14s	remaining: 1m 12s
419:	learn: 0.4209774	total: 6m 15s	remaining: 1m 11s
420:	learn: 0.4208510	total: 6m 16s	remaining: 1m 10s
421:	learn: 0.4206545	total: 6m 17s	remaining: 1m 9s
422:	learn: 0.4204468	total: 6m 18s	remaining: 1m 8s
423:	learn: 0.4202763	total: 6m 19s	remaining: 1m 7s
424:	learn: 0.4201238	total: 6m 20s	remaining: 1m 7s
425:	learn: 0.4197706	total: 6m 20s	re

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.81,-1.12,-1.11,-1.66,-1.06,-2.71
1,-2.96,-1.10,-2.05,-1.71,-1.06,-2.76
2,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
3,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
4,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
5,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
6,-2.67,-0.90,-1.50,-1.43,-1.06,-2.06
7,-2.67,-0.90,-1.50,-1.43,-1.06,-2.06
8,-2.49,-0.97,-1.42,-1.53,-1.07,-2.59
9,-2.49,-0.97,-1.42,-1.53,-1.07,-2.59


In [77]:
# 儲存預測結果(train)
with open('../Output/predict_y_train(Albert).pickle', 'wb') as f:
    pickle.dump(predict_y_train, f)

# 儲存Model
with open('../model/model_Y0(Albert).pickle', 'wb') as f:
    pickle.dump(model_Y0, f)
    
with open('../model/model_Y1(Albert).pickle', 'wb') as f:
    pickle.dump(model_Y1, f)
    
with open('../model/model_Y2(Albert).pickle', 'wb') as f:
    pickle.dump(model_Y2, f)
    
with open('../model/model_Y3(Albert).pickle', 'wb') as f:
    pickle.dump(model_Y3, f)
    
with open('../model/model_Y4(Albert).pickle', 'wb') as f:
    pickle.dump(model_Y4, f)
    
with open('../model/model_Y5(Albert).pickle', 'wb') as f:
    pickle.dump(model_Y5, f)

In [78]:
# 迴圈依據套用模型計算預測值(Y)，並將預測結果放入X中做為新變數供下一輪模型預測時使用。(最佳模型順序[5, 3, 2, 1, 4, 0])

X_PT = data_x.copy()
predict_y_train = pd.DataFrame({})

for k in range(6):
    
    with open('../model/model_Y'+ str(config.order_list[k]) + '(Albert).pickle', 'rb') as f:
        model = pickle.load(f)
    
    # 儲存預測結果
    predict_y_train[config.data_y_col[config.order_list[k]]] = model.predict(X_PT)
    # 把取得的預測值當作變數放進X
    X_PT[config.data_y_col[config.order_list[k]]] = model.predict(X_PT)
    
    # 清空 model
    del model
    
# 調整欄位順序
predict_y_train = predict_y_train[config.data_y_col]
display(predict_y_train)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.81,-1.12,-1.11,-1.66,-1.06,-2.71
1,-2.96,-1.10,-2.05,-1.71,-1.06,-2.76
2,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
3,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
4,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
5,-2.39,-0.87,-1.23,-1.50,-1.06,-2.06
6,-2.67,-0.90,-1.50,-1.43,-1.06,-2.06
7,-2.67,-0.90,-1.50,-1.43,-1.06,-2.06
8,-2.49,-0.97,-1.42,-1.53,-1.07,-2.59
9,-2.49,-0.97,-1.42,-1.53,-1.07,-2.59


In [79]:
# Load y-scaler model
with open(f'../model/standard_scaler_y.pickle', 'rb') as f:
    scaler_y = pickle.load(f)
    
with open(f'../Data/meta/non_scaled_train_y_after_feature_engineering.pickle', 'rb') as f:
    non_scaled_train_y = pickle.load(f)

In [80]:
display(non_scaled_train_y.reset_index())

,index,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,0,42,56,82,41,73,45
1,1,37,60,58,40,64,42
2,4,57,78,62,51,64,65
3,5,42,72,84,54,56,64
4,6,48,72,92,41,76,48
5,7,49,60,78,33,49,55
6,8,41,72,70,47,67,60
7,9,46,70,74,46,62,60
8,10,35,72,92,47,58,62
9,11,35,56,64,42,70,45


In [81]:
from utils.metric_utils import rmse_score

predictions = scaler_y.inverse_transform(predict_y_train)
targets = non_scaled_train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value     5.29
sensor_point6_i_value     8.37
sensor_point7_i_value    10.06
sensor_point8_i_value     8.58
sensor_point9_i_value    11.14
sensor_point10_i_value    7.51
dtype: float64
